This notebook was created to support the data preparation required to support our CS 598 DLH project.  The paper we have chosen for the reproducibility project is:
***Ensembling Classical Machine Learning and Deep Learning Approaches for Morbidity Identification from Clinical Notes ***

This notebook is for creating the multiple embeddings formats as described in the study.

 

The data cannot be shared publicly due to the agreements required to obtain the data so we are storing the data locally and not putting in GitHub.

We are only creating embeddings for data that includes stop words.

In [69]:
pip install torchtext

Note: you may need to restart the kernel to use updated packages.


In [70]:
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize

DATA_PATH = './obesity_data/'

#Field to tokenize on
#tokenize_field = 'lower_text'
tokenize_field = 'tok_lem_text'

#Don't need to do this for the one with no stop words
alldocs_df = pd.read_pickle(DATA_PATH + '/alldocs_df.pkl')
allannot_df= pd.read_pickle(DATA_PATH + '/allannot_df.pkl')

#alldocs_df['word_count'] = alldocs_df[tokenize_field].apply(lambda x: len(x.split()))
#alldocs_df['word_count'] = alldocs_df[tokenize_field].apply(lambda x: len(x.split()))
alldocs_df['word_count'] = alldocs_df[tokenize_field].apply(lambda x: len(x))
alldocs_df['word_count'] = alldocs_df[tokenize_field].apply(lambda x: len(x))

alldocs_df['sentence_count'] = alldocs_df['text'].apply(lambda x: len(sent_tokenize(x)))

df_print = pd.DataFrame()
df_print['Min'] = [np.min(alldocs_df['word_count']), np.min(alldocs_df['word_count'])]
df_print['Mean'] = [np.mean(alldocs_df['word_count']), np.mean(alldocs_df['word_count'])]
df_print['Max'] = [np.max(alldocs_df['word_count']), np.max(alldocs_df['word_count'])]
df_print['Std'] = [np.std(alldocs_df['word_count']), np.std(alldocs_df['word_count'])]
df_print['MeanPlusStd'] = round(df_print['Mean'] + df_print['Std'],0)
token_max = int(round(np.max(df_print['MeanPlusStd']),0))

print(df_print)
print('Max Tokens:',token_max)
print('All:', sum(alldocs_df['word_count'] > token_max), "out of", len(alldocs_df))



   Min        Mean   Max         Std  MeanPlusStd
0  113  973.840787  3748  441.912616       1416.0
1  113  973.840787  3748  441.912616       1416.0
Max Tokens: 1416
All: 156 out of 1118


We are going to split these larger text blocks into 2 notes of size max_token or below.  Note, there are 4 notes (1 in test and 3 in train) that are bigger than 2 times x tokens.  For now, we will ignore, but may want to add in later (either loop or have left/middle/right).

In [71]:
alldocs_df_ok = alldocs_df[alldocs_df['word_count'] <= token_max].copy()
alldocs_df_large_left = alldocs_df[alldocs_df['word_count'] > token_max].copy()
alldocs_df_large_right = alldocs_df[alldocs_df['word_count'] > token_max].copy()

#Get the right words and the left words and then concatenate all 3 and recacluate 
#alldocs_df_large_left[tokenize_field] = alldocs_df_large_left[tokenize_field].apply(lambda x: ' '.join([word for word in x.split()[:(token_max-1)]]))
#alldocs_df_large_right[tokenize_field] = alldocs_df_large_right[tokenize_field].apply(lambda x: ' '.join([word for word in x.split()[token_max:(2*token_max)]]))
alldocs_df_large_left[tokenize_field] = alldocs_df_large_left[tokenize_field].apply(lambda x: [word for word in x[:(token_max-1)]])
alldocs_df_large_right[tokenize_field] = alldocs_df_large_right[tokenize_field].apply(lambda x: [word for word in x[token_max:(2*token_max)]])

alldocs_df_expanded = pd.concat([alldocs_df_ok,alldocs_df_large_right,alldocs_df_large_left])
#alldocs_df_expanded['word_count'] = alldocs_df_expanded[tokenize_field].apply(lambda x: len(x.split()))
alldocs_df_expanded['word_count'] = alldocs_df_expanded[tokenize_field].apply(lambda x: len(x))

print('All:', sum(alldocs_df_expanded['word_count'] > token_max), "out of", len(alldocs_df_expanded))

All: 0 out of 1274


We need to create a one hot vector given a vocabulary and pad it with the padding character.

In [72]:
from typing import Union, Iterable
import torchtext, torch, torch.nn.functional as F
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

sentence_max = np.max(alldocs_df['sentence_count'])
print('Max Sentences:', sentence_max)

#corpus = alldocs_df_expanded[tokenize_field]
#tokenizer = get_tokenizer("basic_english")
#tokens = [tokenizer(doc) for doc in corpus]

#voc = build_vocab_from_iterator(alldocs_df_expanded[tokenize_field], specials = ['<pad>'])
voc = build_vocab_from_iterator(alldocs_df_expanded[tokenize_field].to_list(), specials = ['<pad>'])

#need to create one hot encoding but add <pad> to reach max_tokens
def encode_and_pad(vocab, input_tokens, token_max):
    pad_zeros = token_max - len(input_tokens)
    result = vocab.lookup_indices(input_tokens)
    if pad_zeros > 0:
        result.extend(np.zeros(pad_zeros, dtype=int))
    return result

#need to create tokens add <pad> to reach max_tokens
def token_and_pad(vocab, input_tokens, token_max):
    pad_zeros = token_max - len(input_tokens)
    result = input_tokens
    if pad_zeros > 0:
        zeros = []
        for i in range(pad_zeros):
            zeros.append('<pad>')
        result.extend(zeros)
    return result

#need to create tokens add '\n' to reach max_sentences
def token_and_pad_sentence(input_sentences, sentence_max):
    pad_spaces = sentence_max - len(input_sentences)
    result = input_sentences
    if pad_spaces > 0:
        for i in range(pad_spaces):
            result.append('\n')

    return result

#alldocs_df_expanded['one_hot'] = alldocs_df_expanded[tokenize_field].apply(lambda x: encode_and_pad(voc, x.split(), token_max))
#alldocs_df_expanded['vector_tokenized'] = alldocs_df_expanded[tokenize_field].apply(lambda x: token_and_pad(voc, x.split(), token_max))
#alldocs_df_expanded['sentence_tokenized'] = alldocs_df_expanded[tokenize_field].apply(lambda x: token_and_pad_sentence(sent_tokenize(x), sentence_max))
alldocs_df_expanded['one_hot'] = alldocs_df_expanded[tokenize_field].apply(lambda x: encode_and_pad(voc, x, token_max))
alldocs_df_expanded['vector_tokenized'] = alldocs_df_expanded[tokenize_field].apply(lambda x: token_and_pad(voc, x, token_max))
alldocs_df_expanded['sentence_tokenized'] = alldocs_df_expanded['text'].apply(lambda x: token_and_pad_sentence(sent_tokenize(x), sentence_max))

print(alldocs_df_expanded.iloc[0][tokenize_field])
print(alldocs_df_expanded.iloc[0]['one_hot'])
print(alldocs_df_expanded.iloc[0]['vector_tokenized'])
print(alldocs_df_expanded.iloc[0]['sentence_tokenized'])



Max Sentences: 380
['emh', 'am', 'discharge', 'summary', 'signed', 'dis', 'admission', 'date', 'report', 'status', 'signed', 'discharge', 'date', 'principle', 'diagnosis', 'coronary', 'artery', 'disease', 'other', 'diagnosis', 'peripheral', 'vascular', 'disease', 'hypertension', 'allergy', 'no', 'known', 'drug', 'allergy', 'history', 'of', 'present', 'illness', 'the', 'patient', 'is', 'a', 'year', 'old', 'male', 'immigrant', 'from', 'tope', 'ri', 'with', 'a', 'long', 'history', 'of', 'angina', 'he', 'had', 'been', 'followed', 'in', 'the', 'o', 'lake', 'jack', 'for', 'year', 'with', 'strong', 'indication', 'for', 'interventional', 'evaluation', 'of', 'his', 'coronary', 'artery', 'disease', 'the', 'patient', 'had', 'refused', 'and', 'had', 'been', 'being', 'treated', 'medically', 'inspite', 'of', 'the', 'angina', 'pattern', 'recently', 'his', 'angina', 'had', 'worsened', 'and', 'he', 'agreed', 'to', 'undergo', 'more', 'intensive', 'workup', 'he', 'wa', 'referred', 'for', 'elective', 'car

Join the test data documents with their associated annotations.  Verify the number of records are the same.

In [73]:
all_df = pd.merge(allannot_df,alldocs_df, on='id')
all_df_expanded= pd.merge(allannot_df,alldocs_df_expanded,on='id')



print("All:", len(allannot_df), len(alldocs_df), len(all_df))
print("All Expanded:", len(allannot_df), len(alldocs_df_expanded), len(all_df_expanded))



All: 16325 1118 16325
All Expanded: 16325 1274 18584


Try and validate the numbers are close with the original papers.  You can see the counts are higher for some reason but the percentage occurrence of each disease doesn't change too much so we are good to use the expanded set.

In [74]:
df_before = pd.concat([all_df['disease'].value_counts().sort_index(0),all_df[all_df['judgment']==True]['disease'].value_counts().sort_index(0)/all_df['disease'].value_counts().sort_index(0)],axis =1)
df_after = pd.concat([all_df_expanded['disease'].value_counts().sort_index(0),all_df_expanded[all_df_expanded['judgment']==True]['disease'].value_counts().sort_index(0)/all_df_expanded['disease'].value_counts().sort_index(0)],axis =1)
df_all = pd.concat([df_before,df_after], axis=1)

df_all


C:\Users\chkabat\AppData\Local\Temp/ipykernel_20948/4122723602.py:1: FutureWarning: In a future version of pandas all arguments of Series.sort_index will be keyword-only
  df_before = pd.concat([all_df['disease'].value_counts().sort_index(0),all_df[all_df['judgment']==True]['disease'].value_counts().sort_index(0)/all_df['disease'].value_counts().sort_index(0)],axis =1)
C:\Users\chkabat\AppData\Local\Temp/ipykernel_20948/4122723602.py:2: FutureWarning: In a future version of pandas all arguments of Series.sort_index will be keyword-only
  df_after = pd.concat([all_df_expanded['disease'].value_counts().sort_index(0),all_df_expanded[all_df_expanded['judgment']==True]['disease'].value_counts().sort_index(0)/all_df_expanded['disease'].value_counts().sort_index(0)],axis =1)


,disease,disease,disease,disease
Asthma,1057,0.142857,1200,0.143333
CAD,1044,0.606322,1192,0.608221
CHF,723,0.672199,841,0.693222
Depression,1068,0.220974,1216,0.231086
Diabetes,1070,0.702804,1221,0.719902
GERD,924,0.239177,1039,0.246391
Gallstones,1097,0.164084,1249,0.172938
Gout,1102,0.131579,1255,0.136255
Hypercholesterolemia,961,0.548387,1092,0.553114
Hypertension,1037,0.812922,1182,0.816413


![Note occurrences](images\note_occurrences.gif)


Save the final test/train dataset

In [75]:
all_df.to_pickle(DATA_PATH + '/all_df.pkl') 
all_df_expanded.to_pickle(DATA_PATH + '/all_df_expanded.pkl') 
#corpus.to_pickle(DATA_PATH + '/corpus.pkl')

Testing the GloVe embedding

In [76]:
from torchtext.vocab import vocab


vec = torchtext.vocab.GloVe(name='6B', dim=300)

one_hot_test = all_df_expanded.iloc[0]['one_hot']
vector_tokenized_test = all_df_expanded.iloc[0]['vector_tokenized']

print(one_hot_test)
ret = vec.get_vecs_by_tokens(voc.lookup_tokens(one_hot_test))
print(ret.shape)
#print(ret[0])
print(vector_tokenized_test)
ret = vec.get_vecs_by_tokens(voc.lookup_tokens(one_hot_test))
print(ret.shape)
#print(ret[0])



[213, 7, 17, 75, 8, 51, 7189, 26787, 57, 24369, 7, 17, 75, 2, 51, 7934, 23167, 7, 17, 75, 163, 35, 1, 10, 20, 220, 163, 2, 16, 1758, 2375, 20, 16, 1044, 5849, 21610, 280, 161, 449, 4988, 21797, 32, 576, 570, 1765, 754, 134, 23, 13763, 5237, 102, 8910, 7934, 6938, 431, 314, 118, 126, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Testing the FastText embedding.

In [77]:
from torchtext.vocab import vocab

vec = torchtext.vocab.FastText()

one_hot_test = all_df_expanded.iloc[0]['one_hot']

print(one_hot_test)
ret = vec.get_vecs_by_tokens(voc.lookup_tokens(one_hot_test))
print(ret.shape)

[213, 7, 17, 75, 8, 51, 7189, 26787, 57, 24369, 7, 17, 75, 2, 51, 7934, 23167, 7, 17, 75, 163, 35, 1, 10, 20, 220, 163, 2, 16, 1758, 2375, 20, 16, 1044, 5849, 21610, 280, 161, 449, 4988, 21797, 32, 576, 570, 1765, 754, 134, 23, 13763, 5237, 102, 8910, 7934, 6938, 431, 314, 118, 126, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,